# About this notebook
- Deberta-base starter code
- pip wheels is [here](https://www.kaggle.com/yasufuminakama/nbme-pip-wheels)
- Training notebook is [here](https://www.kaggle.com/yasufuminakama/nbme-deberta-base-baseline-train)

If this notebook is helpful, feel free to upvote :)

In [1]:
m = to_set(1,2,3
print(type(m))

AttributeError: 'tuple' object has no attribute 'to_set'

In [1]:
# The following is necessary if you want to use the fast tokenizer for deberta v2 or v3
# This must be done before importing transformers
import shutil
from pathlib import Path

#transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")
#transformers_path = Path('E:\\Apps\\anacondas\\lib\\site-packages\\transformers')
transformers_path = Path('/home/artem/anaconda3/envs/base_env0/lib/python3.9/site-packages/transformers')


#input_dir = Path("../input/deberta-v2-3-fast-tokenizer")
#input_dir = Path("E:/programming/jupyter_files/kaggle/NBME_patient_notes/debetra_tokenizer")
input_dir = Path("/home/artem/jupyter_files/NBME_patient_notes/debetra_tokenizer")



convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py']:
    filepath = deberta_v2_path/filename
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)


# CFG

In [2]:
# ====================================================
# CFG
# ====================================================
class CFG:
    num_workers=4
    path="/home/artem/jupyter_files/NBME_patient_notes/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=4
    fc_dropout=0.2
    max_len=512
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    gradient_accumulation_steps=1
    print_freq=100

# Library

In [3]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

#os.system('pip uninstall -y transformers')
#os.system('python -m pip install --no-index --find-links=../input/nbme-pip-wheels transformers')
#os.system('pip install tokenizers')
#os.system('pip install transformers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true
from IPython.display import clear_output

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(torch.device('cuda:1'))
print(torch.cuda.get_device_name())

tokenizers.__version__: 0.11.6
transformers.__version__: 4.17.0
env: TOKENIZERS_PARALLELISM=true
Tesla K40m


# tokenizer

In [4]:
# ====================================================
# tokenizer
# ====================================================
#CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')


from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast
tokenizer = DebertaV2TokenizerFast.from_pretrained(CFG.path+'tokenizer/')

#tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

# Helper functions for scoring

In [5]:
# From https://www.kaggle.com/theoviel/evaluation-metric-folds-baseline

def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds)


def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


In [6]:
def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df)
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')
    # create labels
    truths = []
    for location_list in df['location_for_create_labels'].values:
        truth = []
        if len(location_list) > 0:
            location = location_list[0]
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
    return truths


def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
    return results


def get_results(char_probs, th=0.5):
    results = []
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
    return results



def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
    return predictions

# Utils

In [7]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = span_micro_f1(y_true, y_pred)
    return score


def get_logger(filename='inference'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# OOF

# Data Loading

In [8]:
# ====================================================
# Data Loading
# ====================================================
#train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
train = pd.read_csv('/home/artem/jupyter_files/NBME_patient_notes/Data/train.csv')
#train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/NBME_patient_notes2/Data/train.csv')
train['annotation'] = train['annotation'].apply(ast.literal_eval)
train['location'] = train['location'].apply(ast.literal_eval)
#features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
features = pd.read_csv('/home/artem/jupyter_files/NBME_patient_notes/Data/features.csv')
#features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/NBME_patient_notes2/Data/features.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features
features = preprocess_features(features)
#patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')
patient_notes = pd.read_csv('/home/artem/jupyter_files/NBME_patient_notes/Data/patient_notes.csv')
#patient_notes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/NBME_patient_notes2/Data/patient_notes.csv')

print(f"train.shape: {train.shape}")
display(train.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

train.shape: (14300, 6)


,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724]
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693]
2,00016_002,0,16,2,[chest pressure],[203 217]
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]"
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258]


features.shape: (143, 3)


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded


patient_notes.shape: (42146, 3)


,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [9]:
train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(train.head())

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,HPI: 17yo M presents with palpitations. Patien...


In [10]:
# incorrect annotation
train.loc[338, 'annotation'] = ast.literal_eval('[["father heart attack"]]')
train.loc[338, 'location'] = ast.literal_eval('[["764 783"]]')

train.loc[621, 'annotation'] = ast.literal_eval('[["for the last 2-3 months"]]')
train.loc[621, 'location'] = ast.literal_eval('[["77 100"]]')

train.loc[655, 'annotation'] = ast.literal_eval('[["no heat intolerance"], ["no cold intolerance"]]')
train.loc[655, 'location'] = ast.literal_eval('[["285 292;301 312"], ["285 287;296 312"]]')

train.loc[1262, 'annotation'] = ast.literal_eval('[["mother thyroid problem"]]')
train.loc[1262, 'location'] = ast.literal_eval('[["551 557;565 580"]]')

train.loc[1265, 'annotation'] = ast.literal_eval('[[\'felt like he was going to "pass out"\']]')
train.loc[1265, 'location'] = ast.literal_eval('[["131 135;181 212"]]')

train.loc[1396, 'annotation'] = ast.literal_eval('[["stool , with no blood"]]')
train.loc[1396, 'location'] = ast.literal_eval('[["259 280"]]')

train.loc[1591, 'annotation'] = ast.literal_eval('[["diarrhoe non blooody"]]')
train.loc[1591, 'location'] = ast.literal_eval('[["176 184;201 212"]]')

train.loc[1615, 'annotation'] = ast.literal_eval('[["diarrhea for last 2-3 days"]]')
train.loc[1615, 'location'] = ast.literal_eval('[["249 257;271 288"]]')

train.loc[1664, 'annotation'] = ast.literal_eval('[["no vaginal discharge"]]')
train.loc[1664, 'location'] = ast.literal_eval('[["822 824;907 924"]]')

train.loc[1714, 'annotation'] = ast.literal_eval('[["started about 8-10 hours ago"]]')
train.loc[1714, 'location'] = ast.literal_eval('[["101 129"]]')

train.loc[1929, 'annotation'] = ast.literal_eval('[["no blood in the stool"]]')
train.loc[1929, 'location'] = ast.literal_eval('[["531 539;549 561"]]')

train.loc[2134, 'annotation'] = ast.literal_eval('[["last sexually active 9 months ago"]]')
train.loc[2134, 'location'] = ast.literal_eval('[["540 560;581 593"]]')

train.loc[2191, 'annotation'] = ast.literal_eval('[["right lower quadrant pain"]]')
train.loc[2191, 'location'] = ast.literal_eval('[["32 57"]]')

train.loc[2553, 'annotation'] = ast.literal_eval('[["diarrhoea no blood"]]')
train.loc[2553, 'location'] = ast.literal_eval('[["308 317;376 384"]]')

train.loc[3124, 'annotation'] = ast.literal_eval('[["sweating"]]')
train.loc[3124, 'location'] = ast.literal_eval('[["549 557"]]')

train.loc[3858, 'annotation'] = ast.literal_eval('[["previously as regular"], ["previously eveyr 28-29 days"], ["previously lasting 5 days"], ["previously regular flow"]]')
train.loc[3858, 'location'] = ast.literal_eval('[["102 123"], ["102 112;125 141"], ["102 112;143 157"], ["102 112;159 171"]]')

train.loc[4373, 'annotation'] = ast.literal_eval('[["for 2 months"]]')
train.loc[4373, 'location'] = ast.literal_eval('[["33 45"]]')

train.loc[4763, 'annotation'] = ast.literal_eval('[["35 year old"]]')
train.loc[4763, 'location'] = ast.literal_eval('[["5 16"]]')

train.loc[4782, 'annotation'] = ast.literal_eval('[["darker brown stools"]]')
train.loc[4782, 'location'] = ast.literal_eval('[["175 194"]]')

train.loc[4908, 'annotation'] = ast.literal_eval('[["uncle with peptic ulcer"]]')
train.loc[4908, 'location'] = ast.literal_eval('[["700 723"]]')

train.loc[6016, 'annotation'] = ast.literal_eval('[["difficulty falling asleep"]]')
train.loc[6016, 'location'] = ast.literal_eval('[["225 250"]]')

train.loc[6192, 'annotation'] = ast.literal_eval('[["helps to take care of aging mother and in-laws"]]')
train.loc[6192, 'location'] = ast.literal_eval('[["197 218;236 260"]]')

train.loc[6380, 'annotation'] = ast.literal_eval('[["No hair changes"], ["No skin changes"], ["No GI changes"], ["No palpitations"], ["No excessive sweating"]]')
train.loc[6380, 'location'] = ast.literal_eval('[["480 482;507 519"], ["480 482;499 503;512 519"], ["480 482;521 531"], ["480 482;533 545"], ["480 482;564 582"]]')

train.loc[6562, 'annotation'] = ast.literal_eval('[["stressed due to taking care of her mother"], ["stressed due to taking care of husbands parents"]]')
train.loc[6562, 'location'] = ast.literal_eval('[["290 320;327 337"], ["290 320;342 358"]]')

train.loc[6862, 'annotation'] = ast.literal_eval('[["stressor taking care of many sick family members"]]')
train.loc[6862, 'location'] = ast.literal_eval('[["288 296;324 363"]]')

train.loc[7022, 'annotation'] = ast.literal_eval('[["heart started racing and felt numbness for the 1st time in her finger tips"]]')
train.loc[7022, 'location'] = ast.literal_eval('[["108 182"]]')

train.loc[7422, 'annotation'] = ast.literal_eval('[["first started 5 yrs"]]')
train.loc[7422, 'location'] = ast.literal_eval('[["102 121"]]')

train.loc[8876, 'annotation'] = ast.literal_eval('[["No shortness of breath"]]')
train.loc[8876, 'location'] = ast.literal_eval('[["481 483;533 552"]]')

train.loc[9027, 'annotation'] = ast.literal_eval('[["recent URI"], ["nasal stuffines, rhinorrhea, for 3-4 days"]]')
train.loc[9027, 'location'] = ast.literal_eval('[["92 102"], ["123 164"]]')

train.loc[9938, 'annotation'] = ast.literal_eval('[["irregularity with her cycles"], ["heavier bleeding"], ["changes her pad every couple hours"]]')
train.loc[9938, 'location'] = ast.literal_eval('[["89 117"], ["122 138"], ["368 402"]]')

train.loc[9973, 'annotation'] = ast.literal_eval('[["gaining 10-15 lbs"]]')
train.loc[9973, 'location'] = ast.literal_eval('[["344 361"]]')

train.loc[10513, 'annotation'] = ast.literal_eval('[["weight gain"], ["gain of 10-16lbs"]]')
train.loc[10513, 'location'] = ast.literal_eval('[["600 611"], ["607 623"]]')

train.loc[11551, 'annotation'] = ast.literal_eval('[["seeing her son knows are not real"]]')
train.loc[11551, 'location'] = ast.literal_eval('[["386 400;443 461"]]')

train.loc[11677, 'annotation'] = ast.literal_eval('[["saw him once in the kitchen after he died"]]')
train.loc[11677, 'location'] = ast.literal_eval('[["160 201"]]')

train.loc[12124, 'annotation'] = ast.literal_eval('[["tried Ambien but it didnt work"]]')
train.loc[12124, 'location'] = ast.literal_eval('[["325 337;349 366"]]')

train.loc[12279, 'annotation'] = ast.literal_eval('[["heard what she described as a party later than evening these things did not actually happen"]]')
train.loc[12279, 'location'] = ast.literal_eval('[["405 459;488 524"]]')

train.loc[12289, 'annotation'] = ast.literal_eval('[["experienced seeing her son at the kitchen table these things did not actually happen"]]')
train.loc[12289, 'location'] = ast.literal_eval('[["353 400;488 524"]]')

train.loc[13238, 'annotation'] = ast.literal_eval('[["SCRACHY THROAT"], ["RUNNY NOSE"]]')
train.loc[13238, 'location'] = ast.literal_eval('[["293 307"], ["321 331"]]')

train.loc[13297, 'annotation'] = ast.literal_eval('[["without improvement when taking tylenol"], ["without improvement when taking ibuprofen"]]')
train.loc[13297, 'location'] = ast.literal_eval('[["182 221"], ["182 213;225 234"]]')

train.loc[13299, 'annotation'] = ast.literal_eval('[["yesterday"], ["yesterday"]]')
train.loc[13299, 'location'] = ast.literal_eval('[["79 88"], ["409 418"]]')

train.loc[13845, 'annotation'] = ast.literal_eval('[["headache global"], ["headache throughout her head"]]')
train.loc[13845, 'location'] = ast.literal_eval('[["86 94;230 236"], ["86 94;237 256"]]')

train.loc[14083, 'annotation'] = ast.literal_eval('[["headache generalized in her head"]]')
train.loc[14083, 'location'] = ast.literal_eval('[["56 64;156 179"]]')

In [11]:
#download extra datasets: fixed typos, full abbreviation
extra_train = pd.read_pickle('/home/artem/jupyter_files/NBME_patient_notes/typos/extra_train.pkl')
train_full = pd.concat([train,extra_train],ignore_index = True)

In [12]:
train['annotation_length'] = train['annotation'].apply(len)
train_full['annotation_length'] = train_full['annotation'].apply(len)
display(train['annotation_length'].value_counts())

1    8185
0    4399
2    1292
3     287
4      99
5      27
6       9
7       1
8       1
Name: annotation_length, dtype: int64

In [13]:
# ====================================================
# CV split
# ====================================================
Fold = GroupKFold(n_splits=CFG.n_fold)
groups = train_full['pn_num'].values
for n, (train_index, val_index) in enumerate(Fold.split(train_full, train_full['location'], groups)):
    train_full.loc[val_index, 'fold'] = int(n)
train_full['fold'] = train_full['fold'].astype(int)
display(train_full.groupby('fold').size())
train['fold'] = train_full['fold'].to_numpy()[:len(train)]

fold
0    2955
1    2955
2    2955
3    2955
4    2955
dtype: int64

# Dataset

In [14]:
# ====================================================
# Define max_len
# ====================================================
for text_col in ['pn_history']:
    pn_history_lengths = []
    tk0 = tqdm(train_full[text_col].fillna("").values, total=len(train_full))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        pn_history_lengths.append(length)
    LOGGER.info(f'{text_col} max(lengths): {max(pn_history_lengths)}')

for text_col in ['feature_text']:
    features_lengths = []
    tk0 = tqdm(features[text_col].fillna("").values, total=len(features))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        features_lengths.append(length)
    LOGGER.info(f'{text_col} max(lengths): {max(features_lengths)}')

CFG.max_len = max(pn_history_lengths) + max(features_lengths) + 3 # cls & sep & sep
LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/14775 [00:00<?, ?it/s]

pn_history max(lengths): 309


  0%|          | 0/143 [00:00<?, ?it/s]

feature_text max(lengths): 28
max_len: 340


In [15]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text, feature_text):
    inputs = cfg.tokenizer(text, feature_text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


def create_label(cfg, text, annotation_length, location_list):
    encoded = cfg.tokenizer(text,
                            add_special_tokens=True,
                            max_length=CFG.max_len,
                            padding="max_length",
                            return_offsets_mapping=True)
    offset_mapping = encoded['offset_mapping']
    ignore_idxes = np.where(np.array(encoded.sequence_ids()) != 0)[0]
    label = np.zeros(len(offset_mapping))
    label[ignore_idxes] = -1
    if annotation_length != 0:
        for location in location_list:
            for loc in [s.split() for s in location.split(';')]:
                start_idx = -1
                end_idx = -1
                start, end = int(loc[0]), int(loc[1])
                for idx in range(len(offset_mapping)):
                    if (start_idx == -1) & (start < offset_mapping[idx][0]):
                        start_idx = idx - 1
                    if (end_idx == -1) & (end <= offset_mapping[idx][1]):
                        end_idx = idx + 1
                if start_idx == -1:
                    start_idx = end_idx
                if (start_idx != -1) & (end_idx != -1):
                    label[start_idx:end_idx] = 1
    return torch.tensor(label, dtype=torch.float)

class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, 
                               self.pn_historys[item], 
                               self.feature_texts[item])
        return inputs

class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values
        #self.annotation_lengths = df['annotation_length'].values
        #self.locations = df['location'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, 
                               self.pn_historys[item], 
                               self.feature_texts[item])
        '''
        label = create_label(self.cfg, 
                             self.pn_historys[item], 
                             self.annotation_lengths[item], 
                             self.locations[item])'''
        return inputs, label

# Model

In [16]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

# inference

In [17]:
# ====================================================
# inference
# ====================================================
def inference_fn(valid_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(valid_loader, total=len(valid_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    preds = []
    start = end = time.time()
    for step, inputs in enumerate(valid_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        #labels = labels.to(device)
        #batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
            '''
        #loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        #loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps'''
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        end = time.time()
        
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  #'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return predictions

In [18]:
def MistakesProcessingFold0(df,preds):
    new_preds = [[] for i in range(len(preds))]
    for i,df_ind in enumerate(list(df.index)):
        #print(i,df_ind)
        if len(preds[i]) != 0:
            
            for one_loc in preds[i]:
                start,end = one_loc[0],one_loc[1]
                #print(start,end)
                history = df.loc[df_ind]['pn_history']
                if history[start]+history[start+1] == '\\n':
                    start +=2
                else:
                    current = start
                    while current != 0:
                        current -= 1
                        if history[current].isdigit() or history[current].isalpha():
                            start -= 1
                        else:
                            break
                #print(history[start])
                if history[start] == 'n':
                    if start != 0:
                        if history[start-1] == '\\':
                            start +=1
                last_step = start
                found = False
                for match in re.finditer(r"\\n", history[start:end]):
                    #print(history[start:end])
                    found = True
                    index = match.start()
                    print(history[start:end])
                    print('index', index,'start',start)
                    dist_holder = 2 
                    while history[index+start-1].isdigit() == False and history[index+start-1].isalpha() == False:
                        index -= 1
                        dist_holder += 1
                    new_preds[i].append([last_step,index+start])
                    last_step = start +index+dist_holder
                    print(last_step)
                    #value = match.group()
                if found == True:
                    if last_step < end:
                        new_preds[i].append([last_step,end])
                else:
                    print(start,end)
                    new_preds[i].append([start,end])
                    
    return new_preds


def MistakesProcessingAnswers(df,answers):
    new_preds = ['' for i in range(len(preds))]
    
    for i,df_ind in enumerate(list(df.index)):
        #print(i,df_ind)
        if answers[i] != "":
            for one_loc in [s.split() for s in answers[i].split(';')]:
                start,end = int(one_loc[0]),int(one_loc[1])
                #print(start,end)
                history = df.loc[df_ind]['pn_history']
                if history[start]+history[start+1] == '\\n':
                    start +=2
                else:
                    current = start
                    while current != 0:
                        current -= 1
                        if history[current].isdigit() or history[current].isalpha():
                            start -= 1
                        else:
                            break
                #print(history[start])
                if history[start] == 'n':
                    if start != 0:
                        if history[start-1] == '\\':
                            start +=1
                last_step = start
                found = False
                for match in re.finditer(r"\\n", history[start:end]):
                    #print(history[start:end])
                    found = True
                    index = match.start()
                    dist_holder = 2 
                    while history[index+start-1].isdigit() == False and history[index+start-1].isalpha() == False:
                        index -= 1
                        dist_holder += 1
                    if new_preds[i] != '':
                        new_preds[i] += ';' + ' '.join(list(map(str,[last_step,index+start])))
                    else:
                        new_preds[i] += ' '.join(list(map(str,[last_step,index+start])))

                    last_step = start +index+dist_holder
                    #value = match.group()
                if found == True:
                    if last_step < end:
                        if new_preds[i] != '':
                            new_preds[i] += ';' + ' '.join(list(map(str,[last_step,end])))
                        else:
                            new_preds[i] += ' '.join(list(map(str,[last_step,end])))
                else:
                    if new_preds[i] != '':
                        new_preds[i] += ';' + ' '.join(list(map(str,[start,end])))
                    else:
                        new_preds[i] += ' '.join(list(map(str,[start,end])))    
    return new_preds

In [19]:
#import pandas as pd
unlabeled = pd.read_csv('/home/artem/jupyter_files/NBME_patient_notes/unlabeled.csv')
display(unlabeled.head())

,pn_num,case_num,pn_history,feature_num,feature_text
0,1646,0,"a 17-year-old male, has come to the student he...",0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1646,0,"a 17-year-old male, has come to the student he...",1,Family-history-of-thyroid-disorder
2,1646,0,"a 17-year-old male, has come to the student he...",2,Chest-pressure
3,1646,0,"a 17-year-old male, has come to the student he...",3,Intermittent-symptoms
4,1646,0,"a 17-year-old male, has come to the student he...",4,Lightheaded


In [20]:
folds = [[4,5,7],[3],[6],[1,9],[0,4,8]]
unlabeled_df = pd.DataFrame(columns = ['pn_num','case_num','pn_history','feature_num','feature_text','locations'])

predictions = []
predictions_post = []
for fold in CFG.trn_fold:
    valid_folds = unlabeled[unlabeled['case_num'].isin(folds[fold])].reset_index(drop=True)
    valid_texts = valid_folds['pn_history'].values
    #valid_labels = create_labels_for_scoring(valid_folds)
    valid_dataset = TestDataset(CFG, valid_folds)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=True)
    #model = CustomModel(CFG, config_path=None, pretrained=True)
    
    checkpoint = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model'])
    model.to(device)
    #prediction = inference_fn(test_loader, model, device)
    criterion = nn.BCEWithLogitsLoss(reduction="none")
    #prediction = valid_fn(valid_loader, model, criterion, device)
    prediction = inference_fn(valid_loader, model, device)
    prediction = prediction.reshape((len(valid_folds), CFG.max_len))
    char_probs = get_char_probs(valid_texts, prediction, CFG.tokenizer)
    #char_probs = get_char_probs(test['pn_history'].values, prediction, CFG.tokenizer)
    results = get_results(char_probs, th=0.5)
    preds = get_predictions(results)
    predictions.append(preds)
    new_results = MistakesProcessingAnswers(valid_folds,results)
    #score_base = get_score(valid_labels, preds)
    #new_preds = MistakesProcessingFold0(valid_folds,preds_np)
    preds_np = np.array(new_results)
    with open(f'preds_fold{fold}_unlabeled.npy','wb') as f:
        np.save(f, preds_np)
    predictions_post.append(new_results)
    current_df = valid_folds.copy()
    current_df['locations'] = list(map(lambda x : [x],new_results))
    unlabeled_df = pd.concat([unlabeled_df,current_df],ignore_index = True)
    #score_new = get_score(valid_labels, new_preds)
    print(f'Fold {fold} finished')
    #predictions.append(char_probs)
    del model, checkpoint, prediction, char_probs; gc.collect()
    torch.cuda.empty_cache()

preds_post = np.array(predictions_post)
with open(f'preds_unlabeld_full.npy','wb') as f:
    np.save(f, preds_post)
unlabeled_df.to_csv('unlabeled_df.csv', index = False)
#predictions = np.mean(predictions, axis=0)

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/555 [00:00<?, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Fold 0 finished


Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[W pthreadpool-cpp.c

...

Fold 3 finished


Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[W pthreadpool-cpp.c

  0%|          | 0/615 [00:00<?, ?it/s]

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Fold 4 finished


In [22]:
display(unlabeled_df)

,pn_num,case_num,pn_history,feature_num,feature_text,locations
0,41791,4,Karin Moore is a 45 y/o female who presents wi...,400,Lack-of-other-thyroid-symptoms,[]
1,41791,4,Karin Moore is a 45 y/o female who presents wi...,401,anxious-OR-nervous,[57 68;131 146]
2,41791,4,Karin Moore is a 45 y/o female who presents wi...,402,Stress-due-to-caring-for-elderly-parents,[]
3,41791,4,Karin Moore is a 45 y/o female who presents wi...,403,Heavy-caffeine-use,[650 678]
4,41791,4,Karin Moore is a 45 y/o female who presents wi...,404,No-depressed-mood,[]
...,...,...,...,...,...,...
8155,81636,8,67 year old female presenting for insomnia. Pa...,813,Visual-hallucination-once,[517 555]
8156,81636,8,67 year old female presenting for insomnia. Pa...,814,FHx-of-depression-OR-Family-history-of-depression,[906 916]
8157,81636,8,67 year old female presenting for insomnia. Pa...,815,Early-wakening,[]
8158,81636,8,67 year old female presenting for insomnia. Pa...,816,No-suicidal-ideations,[356 380]


In [78]:
def getAnnotations(df):
    annots = []
    for ind,row in df.iterrows():
        #print(row[['locations']])
        #print(type(row))
        #print(row[1])
        if row['locations'][0] == '':
            annots.append([])
        else:
            current_annot = ''
            flag = True
            #print(row['locations'])
            for one_loc in [s.split() for s in row['locations'][0].split(';')]:
                start,end = int(one_loc[0]),int(one_loc[1]) 
                if flag:
                    current_annot += row['pn_history'][start:end]
                    flag = False
                else:
                    current_annot += ' ' + row['pn_history'][start:end]
            annots.append([current_annot])
    return annots

def getIds(df):
    ids = []
    for ind,row in df.iterrows():
        pn_len = len(str(row['pn_num']))
        feat_len = len(str(row['feature_num']))
        ids.append('0'*(5-pn_len)+str(row['pn_num'])+'_'+ '0'*(3-feat_len) + str(row['feature_num']))
    return ids

In [74]:
print('0'*2)

00


In [63]:
unlabeled_df_new = unlabeled_df.copy()
unlabeled_df_new['annotations'] = getAnnotations(unlabeled_df)

In [65]:
unlabeled_df_new = unlabeled_df_new.rename(columns={'annotations': "annotation"})

In [80]:
unlabeled_df_new['id'] = getIds(unlabeled_df_new)

In [88]:
cols = unlabeled_df_new.columns.tolist()
cols = [cols[7],cols[1],cols[0],cols[3],cols[6],cols[5],cols[4],cols[2]]
unlabeled_df_new = unlabeled_df_new[cols]



,pn_num,case_num,pn_history,feature_num,feature_text
0,1646,0,"a 17-year-old male, has come to the student he...",0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1646,0,"a 17-year-old male, has come to the student he...",1,Family-history-of-thyroid-disorder
2,1646,0,"a 17-year-old male, has come to the student he...",2,Chest-pressure
3,1646,0,"a 17-year-old male, has come to the student he...",3,Intermittent-symptoms
4,1646,0,"a 17-year-old male, has come to the student he...",4,Lightheaded


In [96]:
unlabeled_df_new = unlabeled_df_new.rename(columns={'locations': "location"})

In [98]:
unlabeled_df_new.to_csv('unlabeled_df.csv', index = False)
unlabeled_df_new.to_pickle('unlabeled_df.pkl')

In [97]:
display(unlabeled_df_new)

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history
0,41791_400,4,41791,400,[],[],Lack-of-other-thyroid-symptoms,Karin Moore is a 45 y/o female who presents wi...
1,41791_401,4,41791,401,[nervousness losing her mind],[57 68;131 146],anxious-OR-nervous,Karin Moore is a 45 y/o female who presents wi...
2,41791_402,4,41791,402,[],[],Stress-due-to-caring-for-elderly-parents,Karin Moore is a 45 y/o female who presents wi...
3,41791_403,4,41791,403,[5-6 cups of coffee every day],[650 678],Heavy-caffeine-use,Karin Moore is a 45 y/o female who presents wi...
4,41791_404,4,41791,404,[],[],No-depressed-mood,Karin Moore is a 45 y/o female who presents wi...
...,...,...,...,...,...,...,...,...
8155,81636_813,8,81636,813,[seeing her deceased son in the kitchen],[517 555],Visual-hallucination-once,67 year old female presenting for insomnia. Pa...
8156,81636_814,8,81636,814,[depression],[906 916],FHx-of-depression-OR-Family-history-of-depression,67 year old female presenting for insomnia. Pa...
8157,81636_815,8,81636,815,[],[],Early-wakening,67 year old female presenting for insomnia. Pa...
8158,81636_816,8,81636,816,[denies suicidal ideation],[356 380],No-suicidal-ideations,67 year old female presenting for insomnia. Pa...


In [67]:
unlabeled_df_new.to_csv('unlabeled_df.csv', index = False)

In [70]:
ds = pd.read_csv('unlabeled_df.csv')
display(ds)

,pn_num,case_num,pn_history,feature_num,feature_text,locations,annotation
0,41791,4,Karin Moore is a 45 y/o female who presents wi...,400,Lack-of-other-thyroid-symptoms,[''],[]
1,41791,4,Karin Moore is a 45 y/o female who presents wi...,401,anxious-OR-nervous,['57 68;131 146'],['nervousness losing her mind']
2,41791,4,Karin Moore is a 45 y/o female who presents wi...,402,Stress-due-to-caring-for-elderly-parents,[''],[]
3,41791,4,Karin Moore is a 45 y/o female who presents wi...,403,Heavy-caffeine-use,['650 678'],['5-6 cups of coffee every day']
4,41791,4,Karin Moore is a 45 y/o female who presents wi...,404,No-depressed-mood,[''],[]
...,...,...,...,...,...,...,...
8155,81636,8,67 year old female presenting for insomnia. Pa...,813,Visual-hallucination-once,['517 555'],['seeing her deceased son in the kitchen']
8156,81636,8,67 year old female presenting for insomnia. Pa...,814,FHx-of-depression-OR-Family-history-of-depression,['906 916'],['depression']
8157,81636,8,67 year old female presenting for insomnia. Pa...,815,Early-wakening,[''],[]
8158,81636,8,67 year old female presenting for insomnia. Pa...,816,No-suicidal-ideations,['356 380'],['denies suicidal ideation']


In [ ]:
for_disp.loc[1161]['pn_history']

In [ ]:
import pandas as pd
import numpy as np
import re
df = pd.DataFrame([r'\nhello, my \nname is Artem'],columns = ['pn_history'])
preds = [[[0,6],[1,15]]]
display(df)

    
print(MistakesProcessingFold0(df,preds))           

In [ ]:
a = r'\nhello, my \nname is Artem'
print(a[1:15])
re.findall(r'\\n',a)

In [ ]:
list(valid_folds.index)

In [ ]:
pred_annots = getAnnotations(valid_folds,preds)
valid_w_preds = valid_folds.copy()
valid_w_preds['annotations_preds'] = pred_annots
valid_w_preds['location_preds'] = preds
#display(valid_w_preds.head())
mistakes_df = valid_w_preds[valid_w_preds['annotation'] != valid_w_preds['annotations_preds']]
print(len(mistakes_df))
for_disp = mistakes_df[['annotation','annotations_preds','location','location_preds','pn_history','feature_text']]

display(for_disp.iloc[600:700])
#display(valid_w_preds.loc[valid_w_preds['annotation'] != valid_w_preds['annotations_preds']])

In [ ]:
display(valid_folds.loc[0]['pn_history'])

In [ ]:
#display(preds)
display(valid_folds)

In [ ]:
scores = [[] for i in range(5)]
for fold in range(5):
    valid_folds = train[train['fold'] == fold].reset_index(drop=True)
    valid_folds['location_preds'] = predictions_post[fold]
    #valid_labels = create_labels_for_scoring(valid_folds)
    for case in range(10):
        df_case = valid_folds[valid_folds['case_num'] == case].reset_index()
        df_labels = create_labels_for_scoring(df_case)
        df_preds = df_case['location_preds'].to_numpy()
        score = get_score(df_labels, df_preds)
        scores[fold].append(score)
for i in range(5):
    print(scores[i])

In [ ]:
scores_np = np.array(scores)
scores_np[:,9]

In [ ]:
display(len(valid_labels))
display(len(preds))
print(valid_labels[0])
print(df_labels[0])
print(preds[0])
print(df_preds)
print(get_score(valid_labels, preds))